# Day 3 - Conversational AI - aka Chatbot!

In [1]:
# imports

import os
from dotenv import load_dotenv
from anthropic import Anthropic
import gradio as gr

/Users/yadavanuj1996/coding/learn-ai/3-LLM-Engineering/backend-ve/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

def print_key_status():
    if anthropic_api_key:
        print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
    else:
        print("Anthropic API Key not set")

print_key_status()

# Initialize Anthropic client
anthropic = Anthropic(api_key=anthropic_api_key)
CLAUDE_MODEL = "claude-3-sonnet-2024-05-14"

system_message = "You are a helpful assistant"

Anthropic API Key exists and begins sk-ant-


# Please read this! :

In the video, I explain how we now need to write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [3]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

# Student Octavio O. has pointed out that this isn't quite as straightforward for Claude -
# see the excellent contribution in community-contributions "Gradio_issue_with_Claude" that handles Claude.

def chat(message, history):
    messages = history + [{"role": "user", "content": message}]
    response = anthropic.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=1024,
        system=system_message,
        messages=messages,
        stream=True,
    )
    partial = ""
    for chunk in response:
        partial += chunk.delta.get("content", "")
        yield partial

## And then enter Gradio's magic!

In [4]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [5]:
gr.ChatInterface(fn=chat, type="messages", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7885
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/yadavanuj1996/coding/learn-ai/3-LLM-Engineering/backend-ve/lib/python3.13/site-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "/Users/yadavanuj1996/coding/learn-ai/3-LLM-Engineering/backend-ve/lib/python3.13/site-packages/gradio/route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "/Users/yadavanuj1996/coding/learn-ai/3-LLM-Engineering/backend-ve/lib/python3.13/site-packages/gradio/blocks.py", line 2235, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "/Users/yadavanuj1996/coding/learn-ai/3-LLM-Engineering/backend-ve/lib/python3.13/site-packages/gradio/blocks.py", line 1758, 

In [6]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [7]:
def chat(message, history):
    messages = history + [{"role": "user", "content": message}]
    response = anthropic.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=1024,
        system=system_message,
        messages=messages,
        stream=True,
    )
    partial = ""
    for chunk in response:
        partial += chunk.delta.get("content", "")
        yield partial

In [8]:
gr.ChatInterface(fn=chat, type="messages", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7886
* To create a public link, set `share=True` in `launch()`.


In [9]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [10]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7887
* To create a public link, set `share=True` in `launch()`.


In [11]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = history + [{"role": "user", "content": message}]
    response = anthropic.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=1024,
        system=relevant_system_message,
        messages=messages,
        stream=True,
    )
    partial = ""
    for chunk in response:
        partial += chunk.delta.get("content", "")
        yield partial

In [12]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.
